<a href="https://colab.research.google.com/github/shivaaugusta/DataAnalyst/blob/main/Data_Biaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install library (kalau belum ada)
!pip install pandas openpyxl

In [2]:
# 2. Import library
import pandas as pd

In [3]:
# 3. Load dataset (.xlsx)
file_path = '/content/dataset_bersih.xlsx'
df = pd.read_excel(file_path)

In [4]:
# 4. (Optional) lihat nama kolom
print(df.columns)

Index(['No_Urut', 'Golongan_Penyusutan', 'Nomor_Rekening',
       'Jenis_Aktiva_Tetap', 'Nomor_Aktiva_Tetap', 'Tahun_Perolehan',
       'Masa_Manfaat_Tahun', 'Tarif_Penyusutan', 'Nilai_Perolehan',
       'Nilai_Buku_Bulan_Lalu', 'Penyusutan_Bulan_Lalu',
       'Biaya_Penyusutan_Bulan', 'Biaya_Penyusutan_Sampai_Bulan',
       'Akumulasi_Penyusutan', 'Nilai_Buku_Bulan_Ini',
       'Nomor_Rekening_Penyusutan', 'Rasio_Penyusutan'],
      dtype='object')


In [6]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [7]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

st.set_page_config(page_title="Analisis Penyusutan Aset", layout="wide")

st.title("📊 Dashboard Analisis Biaya Penyusutan Aset")

# 1️⃣ Upload file
uploaded = st.file_uploader("dataset_bersih.xlsx", type=["xlsx"])

if uploaded:
    df = pd.read_excel(uploaded)

    # ---- 2. Clean Data ----
    # drop baris yang mengandung Subtotal / Total (di kolom object mana pun)
    obj_cols = df.select_dtypes(include='object').columns
    mask = df[obj_cols].apply(lambda x: x.str.contains('Subtotal|Total', case=False, na=False)).any(axis=1)
    df = df[~mask]

    # drop baris kosong
    df = df.dropna(how='all')

    # konversi Tahun_Perolehan
    if pd.api.types.is_datetime64_any_dtype(df['Tahun_Perolehan']):
        df['Tahun_Perolehan'] = df['Tahun_Perolehan'].dt.year

    # isi NaN
    df[obj_cols] = df[obj_cols].fillna('-')
    num_cols = df.select_dtypes(include=['float64','int64']).columns
    df[num_cols] = df[num_cols].fillna(0)
    df.reset_index(drop=True, inplace=True)

    st.subheader("Preview Dataset")
    st.dataframe(df.head())

    # ---- 3. Dashboard Ringkasan ----
    st.subheader("📌 Ringkasan Total")
    col1,col2,col3 = st.columns(3)
    col1.metric("Total Nilai Perolehan", f"{df['Nilai_Perolehan'].sum():,.0f} Rp")
    col2.metric("Total Biaya Penyusutan Bulan", f"{df['Biaya_Penyusutan_Bulan'].sum():,.0f} Rp")
    col3.metric("Total Akumulasi Penyusutan", f"{df['Akumulasi_Penyusutan'].sum():,.0f} Rp")

    # ---- 4. Top 10 Jenis Aktiva ----
    st.subheader("Top 10 Jenis Aktiva Penyumbang Biaya Penyusutan (Bulan)")
    biaya_per_jenis = (
        df.groupby('Jenis_Aktiva_Tetap')['Biaya_Penyusutan_Bulan']
        .sum()
        .sort_values(ascending=False)
    )
    top10_jenis = biaya_per_jenis.head(10)
    max_val = top10_jenis.max()
    divisor = 1e9 if max_val>=1e9 else 1e6
    satuan = "Miliar" if divisor==1e9 else "Juta"
    top10_jenis_scaled = top10_jenis / divisor

    fig1, ax1 = plt.subplots()
    top10_jenis_scaled.plot(kind='bar', ax=ax1)
    ax1.set_ylabel(f'Total Biaya Penyusutan ({satuan} Rp)')
    ax1.set_xlabel("Jenis Aktiva")
    plt.xticks(rotation=45, ha='right')
    st.pyplot(fig1)

    # ---- 5. Top 10 Golongan Penyusutan ----
    st.subheader("Top 10 Golongan Penyusutan berdasarkan Biaya Penyusutan")
    biaya_per_gol = df.groupby('Golongan_Penyusutan')['Biaya_Penyusutan_Bulan'].sum().sort_values(ascending=False)
    top10_gol = biaya_per_gol.head(10) / divisor

    fig2, ax2 = plt.subplots()
    top10_gol.plot(kind='bar', ax=ax2)
    ax2.set_ylabel(f'Total Biaya Penyusutan ({satuan} Rp)')
    ax2.set_xlabel("Golongan Penyusutan")
    plt.xticks(rotation=45, ha='right')
    st.pyplot(fig2)

    # ---- 6. Histogram Rasio Penyusutan ----
    st.subheader("Distribusi Rasio Penyusutan (Biaya Penyusutan / Nilai Buku)")
    # hitung kolom rasio (jika belum ada)
    if 'Rasio_Penyusutan' not in df.columns:
        df['Rasio_Penyusutan'] = df['Biaya_Penyusutan_Sampai_Bulan'] / df['Nilai_Buku_Bulan_Ini']
    rasio_positive = df[df['Rasio_Penyusutan'] > 0]['Rasio_Penyusutan']
    filtered_rasio = rasio_positive[rasio_positive < 20]

    fig3, ax3 = plt.subplots()
    ax3.hist(filtered_rasio, bins=30)
    ax3.set_xlabel("Rasio Penyusutan (<20)")
    ax3.set_ylabel("Jumlah Aset")
    st.pyplot(fig3)

    # ---- 7. Top Aset (Nilai Perolehan dan Nilai Buku) ----
    st.subheader("Top 10 Aset Berdasarkan Nilai Perolehan")
    top10_perol = df.sort_values('Nilai_Perolehan', ascending=False).head(10)
    st.table(top10_perol[['Jenis_Aktiva_Tetap','Nilai_Perolehan']])

    st.subheader("Top 10 Aset Berdasarkan Nilai Buku Bulan Ini")
    top10_book = df.sort_values('Nilai_Buku_Bulan_Ini', ascending=False).head(10)
    st.table(top10_book[['Jenis_Aktiva_Tetap','Nilai_Buku_Bulan_Ini']])

    # ---- 8. Pencarian Keyword (contoh kata "meja") ----
    st.subheader("🔎 Cari Aset berdasarkan Kata Kunci")
    keyword = st.text_input("Masukkan kata kunci (misal: meja, printer, ac, dll)", "")
    if keyword:
        mask = df['Jenis_Aktiva_Tetap'].str.contains(keyword, case=False, na=False)
        filtered = df[mask]
        st.write(f"Jumlah aset yang mengandung '{keyword}':  {len(filtered)}")
        st.write(f"Total nilai perolehan aset tersebut:  {filtered['Nilai_Perolehan'].sum():,.0f} Rp")
        st.dataframe(filtered[['Jenis_Aktiva_Tetap','Nilai_Perolehan','Biaya_Penyusutan_Bulan']])


2025-08-19 08:54:13.661 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-19 08:54:13.662 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-19 08:54:13.848 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-19 08:54:13.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-19 08:54:13.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-19 08:54:13.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-19 08:54:13.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn